# Part 9

Univeral code that we will use for the entire notebook.

In [ ]:
# required libraries
from openai import OpenAI

# libraries needed for streaming output
from typing_extensions import override
from openai import AssistantEventHandler

# additional libraries
import time

In [ ]:
# Create an instance of the OpenAI class
# This assumes you have the OPENAI_API_KEY environment variable set
client = OpenAI()

## Creating Assistants, Threads, and Messages Review
Let's create a new assistant, thread, and some messages for us to use later on and to review the code for creating them.

### Creating an Assistant
First, let's make an Assistant we can use to communicate with our run.

In [ ]:

# Create an assistant.
assistant = client.beta.assistants.create(
    model="gpt-4o",
    instructions="You are a helpful assistant.",
    name="Son of Run Tester Assistant",
    metadata={
        "holds_threads": "True",
        "likes_threads": "True",
        "holds_messages": "True",
        "likes_messages": "True",
    },
    temperature=1,
    top_p=1,
)

# Print the details of the created assistant to check the properties.
print(assistant)
print("\n\n")
print(assistant.name)
print(assistant.metadata)

### Creating a Thread
Now, let's create a Thread that can be used to hold our messages.

In [ ]:
# Create a thread using the OpenAI API and store it in a variable
# The metadata specifies a user identifier
thread = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console
print(thread)

### Creating a Message
Finally, let's create a Message that we can go into the Thread for use later.

In [ ]:
# Create a message in a specific thread using the client's message creation method.
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="Tell me what a penguin is in 100 words or less.",  # The textual content of the message
    metadata={"key": "value"}  # Additional data associated with the message in key-value pairs
)

# Print the entire message object to view its details.
print(message)

# Print a blank line for better readability of the output.
print("\n")

# Print specific attributes of the message.
print(message.id)  # The unique identifier of the message
print(message.content)  # The content of the message
print(message.content[0].text.value)  # Assuming 'content' is a list of text objects, print the value of the first one
print(message.role)  # The role associated with the message

## Creating Streaming Run
Let's create a run and get some output!

In [ ]:
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
# Normally, you would define this class at the top of your script or in a separate file.
class EventHandler(AssistantEventHandler):    
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant text > ", end="", flush=True)
        
    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)
        
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant tool > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
            for output in delta.code_interpreter.outputs:
                if output.type == "logs":
                    print(f"\n{output.logs}", flush=True)

In [7]:
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
    ) as stream:
    stream.until_done()

 endure extreme cold with their dense feathers and a layer of blubber. There are several species of penguins, varying in size and features, such as the Emperor, King, and Adelie penguins.

## Token Management
We have two options for contolling the amount of tokens used:

max_prompt_tokens (integer or null)
Optional
The maximum number of prompt tokens that may be used over the course of the run. The run will make a best effort to use only the number of prompt tokens specified, across multiple turns of the run. If the run exceeds the number of prompt tokens specified, the run will end with status incomplete. 

max_completion_tokens (integer or null)
Optional
The maximum number of completion tokens that may be used over the course of the run. The run will make a best effort to use only the number of completion tokens specified, across multiple turns of the run. If the run exceeds the number of completion tokens specified, the run will end with status incomplete. 

Let's see it in action!

First we start with max_prompt_tokens to limit the input tokens that can be used.

In [8]:
# Let's start by seeing see how many messages we have
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# Initialize a counter for the messages
message_count = 0

# Loop through the messages, print the id and content, and count the messages
for message in messages:
    print(message.id)
    print(message.content)
    print("\n")
    message_count += 1  # Increment the message count

# Print the total number of messages
print(f"Total number of messages: {message_count}")

msg_MAVhSfsYWEP9imHTtk2UrXwT
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird native to the Southern Hemisphere, particularly Antarctica. Known for their distinctive black and white plumage, penguins have streamlined bodies adapted for swimming and waddle awkwardly on land. They are excellent swimmers, using their flippers to propel through water, where they hunt for fish, krill, and squid. Penguins are social birds, often forming large colonies for breeding and nesting. They endure extreme cold with their dense feathers and a layer of blubber. There are several species of penguins, varying in size and features, such as the Emperor, King, and Adelie penguins.'), type='text')]


msg_2FiIEQV0NWq74ZBbiaykFV4J
[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')]


Total number of messages: 2


In [9]:
# First let's list our runs to see what we have
runs = client.beta.threads.runs.list(
    thread_id=thread.id
)

# Dump all the runs
print(runs)
print("\n\n")

# Loop throught the runs and print the id and status
for run in runs:
    print(run.id)
    print(run.status)
    print("\n")

SyncCursorPage[Run](data=[Run(id='run_P9L2GTrEVUSFa8FbWKOuMv6i', assistant_id='asst_rN1GCMyXuzrXoxdcbu979T67', cancelled_at=None, completed_at=1715816083, created_at=1715816080, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=1715816080, status='completed', thread_id='thread_OAMXvqWnBHgPMlziOoOiukBO', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=128, prompt_tokens=44, total_tokens=172), temperature=1.0, top_p=1.0, tool_resources={})], object='list', first_id='run_P9L2GTrEVUSFa8FbWKOuMv6i', last_id='run_P9L2GTrEVUSFa8FbWKOuMv6i', has_more=False)



run_P9L2GTrEVUSFa8FbWKOuMv6i
completed




In [10]:
# Now let's do a run limiting the prompt tokens
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    max_prompt_tokens=5, # must be set to 256 or more
    # max_prompt_tokens=256,
    event_handler=EventHandler(),
    ) as stream:
    stream.until_done()


assistant text > A penguin is a flightless seabird native to the Southern Hemisphere, particularly Antarctica. Known for their black and white plumage, penguins have streamlined bodies adapted for swimming and waddle awkwardly on land. They are excellent swimmers, using their flippers to propel through water, where they hunt for fish, krill, and squid. Penguins are social birds, often forming large colonies for breeding and nesting. They endure extreme cold with their dense feathers and a layer of blubber. Several species exist, varying in size and features, such as the Emperor, King, and Adelie penguins.

And list the runs again

In [11]:
# Let's list our runs to see what we have
runs = client.beta.threads.runs.list(
    thread_id=thread.id
)


# Loop throught the runs and print the id and status
for run in runs:
    print(run.id)
    print(run.status)
    print(run.incomplete_details)
    print("\n")

run_93rVztkwdkaX4EY9CSJWu1FY
incomplete
IncompleteDetails(reason='max_prompt_tokens')


run_P9L2GTrEVUSFa8FbWKOuMv6i
completed
None




And now limiting the completion tokens

In [12]:
# Now let's do a run limiting the completion tokens
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    max_completion_tokens=5, # must be set to 16 or more
    # max_completion_tokens=16,
    event_handler=EventHandler(),
    ) as stream:
    stream.until_done()


assistant text > A penguin is a flightless seabird native to the Southern Hemisphere,

In [13]:
# Now let's list our runs again
runs = client.beta.threads.runs.list(
    thread_id=thread.id
)


# Loop throught the runs and print the id and status
for run in runs:
    print(run.id)
    print(run.status)
    print(run.incomplete_details)
    print("\n")

run_SSgZthpLe616QD6e2NvGD9ct
incomplete
IncompleteDetails(reason='max_completion_tokens')


run_93rVztkwdkaX4EY9CSJWu1FY
incomplete
IncompleteDetails(reason='max_prompt_tokens')


run_P9L2GTrEVUSFa8FbWKOuMv6i
completed
None




Now let's examine truncation_strategy:

truncation_strategy (object)
Optional
Controls for how a thread will be truncated prior to the run. Use this to control the intial context window of the run.

Properties:
type (string)
Required
The truncation strategy to use for the thread. The default is auto. If set to last_messages, the thread will be truncated to the n most recent messages in the thread. When set to auto, messages in the middle of the thread will be dropped to fit the context length of the model, max_prompt_tokens.

last_messages (integer or null)
Optional
The number of most recent messages from the thread when constructing the context for the run.

In [14]:
# Now let's use a truncation strategy
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    max_prompt_tokens=1000,
    truncation_strategy={  # Adding a truncation strategy to control the context of the conversation
        "type": "last_messages",  # Strategy to keep only the most recent messages
        "last_messages": 5  # Keep the last 5 messages in the context window
    },
    event_handler=EventHandler(),
    ) as stream:
    stream.until_done()



assistant text > A penguin is a flightless seabird native to the Southern Hemisphere,

In [15]:
# Now let's use a truncation strategy
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    max_prompt_tokens=256,
    truncation_strategy={  # Adding a truncation strategy to control the context of the conversation
        "type": "auto",  # Strategy to keep only prompt
    },
    event_handler=EventHandler(),
    ) as stream:
    stream.until_done()


assistant text > A penguin is a flightless seabird native to the Southern Hemisphere,

In [16]:
# Let's see how many messages we have now
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# Initialize a counter for the messages
message_count = 0

# Loop through the messages, print the id and content, and count the messages
for message in messages:
    print(message.id)
    print(message.content)
    print("\n")
    message_count += 1  # Increment the message count

# Print the total number of messages
print(f"Total number of messages: {message_count}")


msg_n07urra4ymORoREqVmbyfx2w
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird native to the Southern Hemisphere,'), type='text')]


msg_cXDkEDtEa6fEpN7Ra0YNHINT
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird native to the Southern Hemisphere,'), type='text')]


msg_LCX8KU39URj9UfpDrADUTuMd
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird native to the Southern Hemisphere,'), type='text')]


msg_J5zf4nlPXhcuYQgyCRIJz9rj
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird native to the Southern Hemisphere, particularly Antarctica. Known for their black and white plumage, penguins have streamlined bodies adapted for swimming and waddle awkwardly on land. They are excellent swimmers, using their flippers to propel through water, where they hunt for fish, krill, and squid. Penguins are social birds, often forming large colonies for breeding and nesti

## Controlling Tool Choice and Response Format
In addition to all the other arguments we can modify, we also have tool_choice and response_format:


